In [1]:
from tools.web_tool import WebTool
import time

In [2]:
import os
import json
import openai
from pydantic import BaseModel, Field
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Create client
client = openai.OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key='85e577a7bd21434e2d3f1ab2bd7a2750c6db5eb7ddf09cce131655911c93f622',
)

In [3]:
from typing import List

class LoS(BaseModel):
    "Get List of strings to complete the given quries"
    los: List[str] = Field(description="List of five strings to complete the provided task")

class About(BaseModel):
    "Get description about the company and a timeline of major events and achievements in the company's history"
    description: str = Field(description="Short description about the company.")
    timeline: List[str] = Field(description="List of major events and achievements happed with the company with detailed description, like founded, acquired, went public and other events.")

class Offering(BaseModel):
    "Get products and services offered by the company along with their rough estimated prices"
    offerings: List[str] = Field(description="List of products or services offered by the company along with their rough prices")
    
class SWOTAnalysis(BaseModel):
    "Market share, sales and SWOT (Strength, weakness, opportunity and threat) analysis of given company"
    #numbers : List[str] = Field(description="List of description of key performance indices like market-share, sales, global precense, etc.")
    strength: List[str] = Field(description="List of strenghts the given company have that make them unique, with detailed description")
    weakness: List[str] = Field(description="List of weakness the given company with detailed description")
    opportunities: List[str] = Field(description="List of opportunities the given company have that will contribute in the company future growth with detailed description")
    threats: List[str] = Field(description="List of threats the given company have that can make business go bust, with detailed description")

In [4]:
def get_message(task,**kwargs):
    if task == 'web-query':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your own intelligence to complete the task."},
            {"role": "user", "content": f"Web search queries for DuckDuckGo search engine for '{kwargs['query']}'"}
        ]
    elif task == 'competitors-name':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your own intelligence to complete the task."},
            {"role": "user", "content": f"Name mojor competitors in the {kwargs['industry']} business."}
        ]
    elif task == 'swot-analysis':
        message = [
            {"role": "system", "content": "You are a helpful and experienced market analyst. Use your own intelligence the provided context(if needed) to complete the SWOT analysis for the given company, dont make things up. Dont use Markdown and Answer in valid JSON"},
            {"role": "user", "content": f"Do SWOT analysis on {kwargs['competitor_name']} as a business.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    elif task == 'about':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your own intelligence and the provided context(if needed) to complete the task, dont make things up. Dont use Markdown or other formating and Answer in valid JSON"},
            {"role": "user", "content": f"Tell description about {kwargs['competitor_name']} major events happened with {kwargs['competitor_name']} as a business in correct order.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    elif task == 'offering':
        message = [
            {"role": "system", "content": "You are a helpful assistant. Use your own intelligence and the provided context(if needed) to complete the task, dont make things up. Dont use Markdown or other formating and Answer in valid JSON"},
            {"role": "user", "content": f"Tell few products or services offer by {kwargs['competitor_name']} along with their pricing.\n\nCONTEXT :\n{kwargs['context']}"}
        ]
    else:
        message = [None,None]
    return message

In [5]:
def get_response(response_schema,message,repetetion_penalty):
    print(message)
    response = client.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=message,
        response_format={"type": "json_object", "schema": response_schema.model_json_schema()},
        frequency_penalty = repetetion_penalty
    )
    print(response.choices[0].message.content)
    content = json.loads(response.choices[0].message.content)
    return content

def get_competitor_response(competitor,task,query,schema):
    content = remove_stopwords(WebTool().fetch_content(texts=[query]))[:4096*4]
    clean_content = client.chat.completions.create(
        model='meta-llama/Llama-3-8b-chat-hf',
        messages=[
            {"role": "system", "content": "You are an information retriever and summarizer,ignore everything you know, extract all the factual information regarding the QUERY into a maximum of 400-500 words. Output should be plain text no markdown and chunks of paragraph full of information, ignore links, using the RAW TEXT"},
            {"role": "user", "content": f"QUERY :\n\n{query}\n\n RAW TEXT :\n\n{content}"} 
        ],
        frequency_penalty = 0.08)
    msg = get_message(task=task,competitor_name=competitor,context=clean_content.choices[0].message.content)
    report = get_response(response_schema=schema,message=msg,repetetion_penalty=0.23)

    return report

In [6]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_text = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_text)
def main(industry):
    #get competitors name
    message = get_message(task='competitors-name',industry=industry,context='')
    competitors_names = get_response(response_schema=LoS,message=message,repetetion_penalty=0.5)['los']
    final_report = {}
    for competitor in competitors_names[:2]:
        
        queries = [f'SWOT Strength, weakness, opportunity and threat of {competitor} in {industry} industry.',
                   f'Major events (History) and achievements of {competitor} in {industry} industry',
                   f'Products or services offer by {competitor} in {industry} along with their prices.']

        swot_report = get_competitor_response(competitor=competitor,task='swot-analysis',query=queries[0],schema=SWOTAnalysis)
        final_report[competitor] = swot_report

        about_report = get_competitor_response(competitor=competitor,task='about',query=queries[1],schema=About)
        final_report[competitor].update(about_report)

        products = get_competitor_response(competitor=competitor,task='offering',query=queries[2],schema=Offering)
        final_report[competitor].update(products)

    return final_report

In [12]:
r = main('Footwear')

[{'role': 'system', 'content': 'You are a helpful assistant. Use your own intelligence to complete the task.'}, {'role': 'user', 'content': 'Name mojor competitors in the Footwear business.'}]
 										
{ "los": 
[
"Nike",
"Adidas",
"Puma",
"New Balance",
"Skechers",
"Under Armour",
"Vans",
"Converse",
"Reebok",
"K-Swiss"
]
} 										



/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bstrategyhub.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pestleanalysis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'role': 'system', 'content': 'You are a helpful and experienced market analyst. Use your own intelligence the provided context(if needed) to complete the SWOT analysis for the given company, dont make things up. Dont use Markdown and Answer in valid JSON'}, {'role': 'user', 'content': 'Do SWOT analysis on Nike as a business.\n\nCONTEXT :\nHere is the factual information regarding the query "SWOT Strength, weakness, opportunity and threat of Nike in Footwear industry":\n\nNike is an American multinational corporation that specializes in athletic wear, providing footwear, apparel, athletic equipment, and accessories. The company was founded in 1964 by Bill Bowerman and Phil Knight.\n\nStrengths:\n\n* Strong brand awareness: The swoosh logo is easily recognizable and has a high brand value of $53.7 billion, ranking #9 in the global brand ranking report.\n* Huge customer base: With millions of customers worldwide, loyal to the brand and participating in events and providing feedback.\n* 

/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.businessinsider.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/ashish/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.businessinsider.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ProxyError: HTTPConnectionPool(host='192.158.15.201', port=60684): Max retries exceeded with url: http://www.businessinsider.in/retail/nikes-incredible-road-to-becoming-the-worlds-dominant-sneaker-retailer/articleshow/49220193.cms (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7aef60692290>, 'Connection to 192.158.15.201 timed out. (connect timeout=None)')))

In [8]:
import pandas
pandas.DataFrame(r)

,Nike,Adidas
strength,"[Strong Brand Awareness, Huge Customer Base, A...","[Strong brand value, Innovative product develo..."
weakness,"[Poor Labor Conditions, Retailers Stronger Hol...","[Dependence on global supply chain, Limited pr..."
opportunities,"[Emerging Markets, Innovative Products, Effici...","[Growth in e-commerce, Increasing demand for g..."
threats,"[Counterfeit Products, Increased Competition, ...",[Competition from established brands like Nike...
timeline,[1964: Founding of Blue Ribbon Sports by Bill ...,"[1924: Adidas was founded in Herzogenaurach, B..."
description,Nike has been a leading innovator in athletic ...,This timeline represents major events happened...
offerings,"[Athletic footwear for running, soccer, basket...","[Ultraboost Light Shoes (running), ADIZERO ADI..."


In [44]:
import json
from tools.json2html_report import json2html_report
json_response = json.loads(json.dumps(r))
html_content = json2html_report(json_data=json_response,report_title='competition analysis')
with open('x.html','w') as file:
    file.write(html_content)

JSON converted to HTML report


In [1]:
with open('x.txt','r') as file:
    d = eval(file.read())

In [6]:
d.keys()

dict_keys(['Nike', 'Adidas'])

In [14]:
vars = {
    'competitors':r.keys(),
    'report':r
}

In [15]:
from jinja2 import Environment, FileSystemLoader


In [16]:
env= Environment(loader=FileSystemLoader('templates/'))

In [17]:
temp = env.get_template('competitor.html')

In [21]:
from IPython.core.display import display, HTML
display(HTML(temp.render(vars)))

Strengths,Weaknesses
Strong Brand Awareness Huge Customer Base America's Favorite Sneaker Brand Aimed Sustainability Iconic Relationships Side Brands Low Manufacturing Cost In-house Professionals Superior Marketing Capabilities Black Community Support,Poor Labor Conditions Retailers Stronger Hold Pending Debts Lawsuits Lack of Diversification Contradicting Strategies Dependency on North America Sexual Harassment <!-- Add more weakness points as needed -->
Opportunities,Threats
Emerging Markets Innovative Products Efficient Integration Cutting Ties with Big Retailers Acquisitions Merges with Metaverse Exiting Wholesale Distribution <!-- Add more opportunity points as needed -->,Counterfeit Products Increased Competition Marketing Budget Pressure Currency Foreign Exchange Risks Patent Disputes Economic Uncertainty Trade Tensions <!-- Add more threat points as needed -->
Strengths,Weaknesses
Strong brand value Innovative product development Diversified product portfolio Strategic partnerships Global presence Strong digital presence Social responsibility initiatives,Dependence on global supply chain Limited product line compared to competitors Rising operating expenses High dependence on sports market Outsourced manufacturing risks <!-- Add more weakness points as needed -->
Opportunities,Threats
Growth in e-commerce Increasing demand for gender-neutral collections Expansion in sportswear industry Premium product targeting Smart materials technology application <!-- Add more opportunity points as needed -->,Competition from established brands like Nike & Puma Supplier dominance risk Counterfeit goods issue Economic downturn impacting consumer spending Potential lawsuits related to technology usage without license <!-- Add more threat points as needed -->
